In [ ]:
# default_exp core

# Core

> Main program

The implementation of the main script is actually in the `build` function below. `main` is just a *wrapper* that parses command-line arguments.

In [ ]:
#hide
from nbdev.showdoc import *

In [ ]:
# export

import sys
import argparse
import pathlib
import importlib.util
import string
import collections
from types import ModuleType

import numpy as np
import yaml
from py2gift import util
from py2gift import question

import gift_wrapper.core

# Parsing of the command-line arguments

The function below just parses command-line arguments and pass them to the `build` function below.

In [ ]:
# export

def main():
    
    parser = argparse.ArgumentParser(description='Python to GIFT converter')

    parser.add_argument(
        'input_file', type=argparse.FileType('r'), default='global_settings.yaml', help='settings file', nargs='?')

    parser.add_argument('-c', '--code_directory', default='.', help='directory with the required source code')

    parser.add_argument(
        '-m', '--main_module', default='questions.py', help='file with the questions generators')

    parser.add_argument(
        '-l', '--local', default=False, action='store_true', help="don't try to copy the images to the server")

    command_line_arguments = parser.parse_args()
    
    code_directory = pathlib.Path(command_line_arguments.code_directory)
    main_module = pathlib.Path(command_line_arguments.main_module)

    sys.path.insert(0, code_directory.absolute().as_posix())
    spec = importlib.util.spec_from_file_location(main_module.stem, (code_directory / main_module).absolute())
    questions_generators = importlib.util.module_from_spec(spec)
    spec.loader.exec_module(questions_generators)
    
    build(command_line_arguments.input_file.name, command_line_arguments.local, questions_generators)

# Main script

A function to obtain the initialization parameters of a given class given the corresponding settings.

* `settings`: a `dict` with the required parameters; it should include keys *statement* and *feedback*

It returns another dictionary.

In [ ]:
# export

def init_parameters_from_settings(settings: dict) -> dict:

    init_parameters = {
        'unprocessed_statement': string.Template(settings['statement']),
        'unprocessed_feedback': string.Template(settings['feedback'])
    }

    init_parameters.update(settings.get('init parameters', {}))
    
    return init_parameters

---

A function to build a (single) question from the corresponding settings.

* `category_name`: name of the category
* `class_name`: name of the question's class
* `settings`: a **global** settings dictionary (usually read from a file) in which to find those for the requested category and class
* `module`: an **already imported** Python module in which to find the class named *name*

It returns a `dict`.

In [ ]:
# export

def build_question(question_generator: question.QuestionGenerator, category_name: str, settings: dict) -> dict:
    
    class_name = question_generator.__name__
    
    class_settings = util.extract_class_settings(category_name, class_name, settings)
    
    # an instance
    question_generator = question_generator(**init_parameters_from_settings(class_settings))

    assert ('parameters' in class_settings) ^ ('number of instances' in class_settings), 'either "parameters" or "number of instances" must be specified'

    if 'parameters' in class_settings:
        return question_generator(**class_settings['parameters'])
    else:
        return question_generator()

---

In [ ]:
# export

def build(input_file: str, local_run: bool, questions_module: ModuleType, parameters_file: str = 'parameters.yaml'):

    with open(input_file) as f:

        settings = yaml.load(f, Loader=yaml.FullLoader)

    output_file = settings['output file']

    category_questions = collections.defaultdict(list)

    for cat in settings['categories']:

        questions = []

        for c in cat['classes']:

            this_class_questions = []
            
            assert ('parameters' in c) ^ ('number of instances' in c), 'either "parameters" or "number of instances" must be specified'
            
            question_generator = getattr(questions_module, c['name'])(**init_parameters_from_settings(c))
            
            if 'parameters' in c:

                for p in c['parameters']:

                    this_class_questions.append(question_generator(**p))
            
            else:
                
                for _ in range(c['number of instances']):
                    
                    this_class_questions.append(question_generator())

            questions.extend(util.add_name(this_class_questions, base_name=c['question base name']))

        category_questions[cat['name']].extend(questions)

    # --------

    util.write_multiple_categories(category_questions, settings['pictures base directory'], output_file=output_file)
    
    gift_wrapper.core.wrap(parameters_file, output_file, local_run=local_run, no_checks=False)

This function expects three things (the parameters file is presumed and not an input parameter).

## Module with questions

In [ ]:
class FakeModule:
    
    class TestClass(question.NumericalQuestionGenerator):
    
        def __init__(
            self, unprocessed_statement: string.Template, unprocessed_feedback: string.Template,
            mean: float,
            prng: np.random.RandomState = np.random.RandomState(42)) -> None:

            super().__init__(unprocessed_statement, unprocessed_feedback, prng)
            
            self.mean = mean
        
        def setup(self):

            self.statement = self.unprocessed_statement.substitute(mean=self.mean)

            self.solution = 42
            self.error = '10%'

            self.feedback = self.unprocessed_feedback.substitute()

## Input file

In [ ]:
input_file = 'doc_input.yaml'

In [ ]:
%%writefile {input_file}

output file: third_midterm.yaml
pictures base directory: tc/midterm3
path to gift-wrapper: '~/gift-wrapper/wrap.py'

categories:

  - name: Test category

    classes:

      - name: TestClass

        question base name: Test class
            
        init parameters:
            
            mean: 3

        statement: |
          Consider a Gaussian random variable, $$X$$, with mean $$ \mu = $mean $$ and variance...
          

        feedback: |
          Clearly, $$Y$$ is...
          

        number of instances: 2

## Parameters file

In [ ]:
parameters_file = 'doc_parameters.yaml'

In [ ]:
%%writefile {parameters_file}
images hosting:

  ssh:
    user: mvazquez

    # below, one should specify either a password for the user or  "public key" file but NOT both of them

    password:

    # "~" stands for the user's home directory (in Linux for one...)
    public_key: ~/.ssh/id_rsa_mymachine.pub

  copy:
    # machine into which files will be copied
    host: hidra1

    # the path that in the remote machine acts as root of the publicly visible directories hierarchy (hence it's not
    # visible from outside);  it *should* exist ("." stands for the working directory when you ssh into the machine)
    public filesystem root: ./public_html

  # public address from which the images will hang
  public URL: http://www.tsc.uc3m.es/~mvazquez/

latex:

  # auxiliary TeX file that will be created to check that formulas can be compiled
  auxiliary file: __latex_check.tex

In [ ]:
build(input_file, local_run=True, questions_module=FakeModule, parameters_file=parameters_file)